In [ ]:
import pickle
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from mqt.bench.evaluation import evaluation

font = {"size": 16}
mpl.rc("font", **font)

In [ ]:
evaluation.create_statistics()

In [ ]:
with Path("evaluation_data.pkl").open("rb") as f:
    eval_data = pickle.load(f)
df = pd.DataFrame(eval_data)
df.head()

In [ ]:
df = df.join(pd.json_normalize(df["supermarq_features"])).drop("supermarq_features", axis="columns")
df.head()

In [ ]:
benchmarks = df["filename"].values
num_qubits = df["num_qubits"].values
depth = df["depth"].values
num_gates = df["num_gates"].values
num_multiple_qubit_gates = df["num_multiple_qubit_gates"].values
program_communication = df["program_communication"].values
critical_depth = df["critical_depth"].values
entanglement_ratio = df["entanglement_ratio"].values
parallelism = df["parallelism"].values
liveness = df["liveness"].values
perc_multi = num_multiple_qubit_gates / num_gates

# Device Distribution

In [ ]:
overall_count_mapped = evaluation.count_occurrences(benchmarks, "mapped")
mapped_ibm_montreal_count = evaluation.count_occurrences(benchmarks, "mapped_ibm_montreal")
mapped_ibm_washington_count = evaluation.count_occurrences(benchmarks, "mapped_ibm_washington")
mapped_oqc_lucy_count = evaluation.count_occurrences(benchmarks, "mapped_oqc_lucy")
mapped_rigetti_count = evaluation.count_occurrences(benchmarks, "mapped_rigetti")
mapped_quantinuum_count = evaluation.count_occurrences(benchmarks, "mapped_quantinuum")
mapped_ionq_aria1_count = evaluation.count_occurrences(benchmarks, "mapped_ionq_aria1")
mapped_ionq_harmony_count = evaluation.count_occurrences(benchmarks, "mapped_ionq_harmony")

qiskit_qubit_numbers = evaluation.count_qubit_numbers_per_compiler(benchmarks, "qiskit")
tket_qubit_numbers = evaluation.count_qubit_numbers_per_compiler(benchmarks, "tket")

In [ ]:
perc_montreal = mapped_ibm_montreal_count / overall_count_mapped
perc_washington = mapped_ibm_washington_count / overall_count_mapped
perc_oqc = mapped_oqc_lucy_count / overall_count_mapped
perc_rigetti = mapped_rigetti_count / overall_count_mapped
perc_quantinuum = mapped_quantinuum_count / overall_count_mapped
perc_ionq_aria1 = mapped_ionq_aria1_count / overall_count_mapped
perc_ionq_harmony = mapped_ionq_harmony_count / overall_count_mapped

In [ ]:
font = {"size": 12}
mpl.rc("font", **font)
data = [
    perc_washington,
    perc_rigetti,
    perc_quantinuum,
    perc_montreal,
    perc_ionq_aria1,
    perc_ionq_harmony,
    perc_oqc,
]
colors = ["#B7E6A5", "#7CCBA2", "#46AEA0", "#089099", "#02818B", "#026688", "#025582"]
labels = [
    "IBMQ Washington (127)",
    "Rigetti Aspen M2 (80)",
    "Quantinuum H2 (32)",
    "IBMQ Montreal (27)",
    "IonQ Aria 1 (25)",
    "IonQ Harmony (11)",
    "OQC Lucy (8)",
]
pies = plt.pie(data, labels=labels, autopct="%1.1f%%", colors=colors)
for pie in pies[0]:
    pie.set_alpha(0.7)
plt.savefig("results/pie.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Num Qubits

In [ ]:
font = {"size": 16}
mpl.rc("font", **font)
qiskit_qubit_numbers = evaluation.count_qubit_numbers_per_compiler(benchmarks, "qiskit")
tket_qubit_numbers = evaluation.count_qubit_numbers_per_compiler(benchmarks, "tket")

# colors = ['#7CCBA2', '#46AEA0', '#089099', "#00718B", "#045275", "#003147"]
# colors = ['#089099', '#003147']
plt.figure(figsize=(8, 5))
plot = sns.histplot(
    data=[tket_qubit_numbers, qiskit_qubit_numbers],
    bins=129,
    stat="density",
    alpha=0.3,
    kde=True,
    edgecolor="white",
    linewidth=0.5,
    palette=sns.color_palette("viridis", 2)[::-1],
)
plt.legend(["Qiskit", "TKET"])
plt.ylabel("Relative Frequency")
plt.xlabel("Number of Qubits")
plt.savefig("results/qubit_dist.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Further Characteristics

In [ ]:
datasets = [
    program_communication.astype(float),
    critical_depth.astype(float),
    entanglement_ratio.astype(float),
    liveness.astype(float),
    parallelism.astype(float),
    perc_multi.astype(float),
]
labels = [
    "Program Communication",
    "Critical Depth",
    "Entanglement Ratio",
    "Liveness",
    "Parallelism",
    "Percentage \nMulti-Qubit Gates",
]

In [ ]:
fig = plt.figure(figsize=(19, 4))
ax = fig.add_subplot(111)
colors = ["#7CCBA2", "#46AEA0", "#089099", "#00718B", "#045275", "#003147"]
pos = range(len(datasets))
plots = plt.violinplot(datasets, pos)
for pc, color in zip(plots["bodies"], colors):
    pc.set_facecolor(color)

ax.set_xticks(pos)
plt.xticks(rotation=0)
ax.set_xticklabels(labels)
plt.savefig("results/violins.pdf", format="pdf", bbox_inches="tight")